In [1]:
import pandas as pd
import numpy as np
import string
import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
df=pd.read_csv("../input/mental-health-faq-for-chatbot/Mental_Health_FAQ.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Question_ID  98 non-null     int64 
 1   Questions    98 non-null     object
 2   Answers      98 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [4]:
df.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [5]:
df=df.drop(["Question_ID"],axis="columns")

In [6]:
from nltk.tokenize import word_tokenize
df["Questions"]=df["Questions"].str.lower()

In [7]:
text=df["Questions"][0]

In [8]:
from nltk.tokenize import RegexpTokenizer
text = re.sub(r'[^\w\s]', '', text)
words = word_tokenize(text) 

In [9]:
print(words)

['what', 'does', 'it', 'mean', 'to', 'have', 'a', 'mental', 'illness']


In [10]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
text=" ".join([wnl.lemmatize(t) for t in text.split() ])

In [11]:
text

'what doe it mean to have a mental illness'

In [12]:
from nltk.corpus import stopwords  
stopwords=stopwords.words('english')

In [13]:
def data_prep(text):
  text=text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  text=" ".join(t for t in text.split() if t not in stopwords)
  return text

In [14]:
print(data_prep(text))

doe mean mental illness


In [15]:
df["Questions"]=df["Questions"].apply(data_prep)

In [16]:
df["Questions"]

0               mean mental illness
1             mental illness affect
2             causes mental illness
3      warning signs mental illness
4     people mental illness recover
                  ...              
93            know im drinking much
94    cannabis dangerous legalizing
95          convince kids use drugs
96    legal status evidence cbd oil
97                  evidence vaping
Name: Questions, Length: 98, dtype: object

In [17]:
print(df["Questions"][0])

mean mental illness


In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
#Tfidf vectorizer
tf=TfidfVectorizer()
#transformed train reviews
tf_train=tf.fit_transform(df["Questions"])
# #transformed test reviews
# tf_test=tf.transform(test_reviews_data)
# print('Tfidf_train:',tf_train.shape)
# print('Tfidf_test:',tf_test.shape)


In [20]:
df_check=pd.DataFrame(tf_train.toarray(),columns=tf.get_feature_names())

In [21]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
df["Answers_Code"]=le.fit_transform(df["Answers"])

In [22]:
df["Answers_Code"]

0     48
1     35
2     36
3     72
4     91
      ..
93    69
94    18
95    96
96    17
97     0
Name: Answers_Code, Length: 98, dtype: int64

In [23]:
df

,Questions,Answers,Answers_Code
0,mean mental illness,Mental illnesses are health conditions that di...,48
1,mental illness affect,It is estimated that mental illness affects 1 ...,35
2,causes mental illness,It is estimated that mental illness affects 1 ...,36
3,warning signs mental illness,Symptoms of mental health disorders vary depen...,72
4,people mental illness recover,"When healing from mental illness, early identi...",91
...,...,...,...
93,know im drinking much,Sorting out if you are drinking too much can b...,69
94,cannabis dangerous legalizing,"Cannabis smoke, for example, contains cancer-c...",18
95,convince kids use drugs,You can't. But you can influence their capacit...,96
96,legal status evidence cbd oil,Cannabidiol or CBD is a naturally occurring co...,17


In [24]:
from sklearn.naive_bayes import MultinomialNB
mn=MultinomialNB()
mn.fit(df_check,df["Answers_Code"])

MultinomialNB()

In [25]:
test=[
    "What does mental illness mean?",
    "What are the causes of mental illness?"
]

In [26]:
testing=tf.transform(test)

In [27]:
mn.predict(testing)

array([48, 36])

In [28]:
Ans=df["Answers"].unique()
Ans=Ans.tolist()
Ans_Code=df["Answers_Code"].unique()
Ans_Code=Ans_Code.tolist()

In [29]:
Ans_Code.index(48)
Ans_Code.index(36)

2

In [30]:
Ans[2]

'It is estimated that mental illness affects 1 in 5 adults in America, and that 1 in 24 adults have a serious mental illness. Mental illness does not discriminate; it can affect anyone, regardless of gender, age, income, social status, ethnicity, religion, sexual orientation, or background. Although mental illness can affect anyone, certain conditions may be more common in different populations. For instance, eating disorders tend to occur more often in females, while disorders such as attention deficit/hyperactivity disorder is more prevalent in children. Additionally, all ages are susceptible, but the young and the old are especially vulnerable. Mental illnesses usually strike individuals in the prime of their lives, with 75 percent of mental health conditions developing by the age of 24. This makes identification and treatment of mental disorders particularly difficult, because the normal personality and behavioral changes of adolescence may mask symptoms of a mental health conditio